In [1]:
from tensorflow.keras.models import load_model,Model
from pydub import AudioSegment
import os
import random
import librosa
from spleeter.separator import Separator
import shutil
import gc
import tensorflow as tf
import numpy as np
import json

In [12]:
def extract_instrumental(separator,audio,name):
    # Wait for implementation
    return ""

In [13]:
def extract_audio():
    # Wait for implementation
    return ""

In [14]:
def preprocess():
    # Wait for implementation
    return ""

In [2]:
def get_bpm(path):
    path = 'extract/' + path + '.wav'
    y, sr = librosa.load(path)
    tempo, _ = librosa.beat.beat_track(y,sr)
    del y, sr
    gc.collect()
    return tempo

In [3]:
def get_spectrogram(path):
    path = "spectogram/" + path + ".png"
    IMAGE_SIZE = 224
    img = tf.keras.preprocessing.image.load_img(path, target_size=(IMAGE_SIZE, IMAGE_SIZE)
    )
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    img_array = img_array / 255
    img_array = img_array.reshape(1,IMAGE_SIZE,IMAGE_SIZE,3)
    return img_array

In [4]:
def manipulate_audio(filepath):
    # Wait for implementation
    # extract_instrumental(audio)
    # extract_audio()
    audio_list = []
    for mood in os.listdir("spectogram"):
        for file in os.listdir("spectogram/"+mood):
            audio_list.append(mood+"/"+file.split(".")[0])
    return audio_list

In [5]:
def get_model():
    model_path = 'models/model_v1.h5'
    model = load_model(model_path)
    return model

In [6]:
def prediction(model,audio):
    class_labels = ["Anxious","Chill","Focus","Party","Romance","Sad"]
    predictions = model.predict(audio)
    mood = class_labels[np.argmax(predictions)]   
    feature_model = Model(model.input,model.layers[-6].output)
    features = feature_model.predict(audio)
    return mood,features

In [9]:
filepath = ""
model = get_model()
audios = manipulate_audio(filepath)
data = []
i = 0
for audio in audios:
    spectrogram = get_spectrogram(audio)
    bpm = get_bpm(audio)
    mood, features = prediction(model,spectrogram)
    data.append({
        "music_name":audio.split("/")[1],
        "music_path": "song/" + audio + ".wav",
        "music_feature_path": "features/" + audio.split("/")[1] + ".json",
        "bpm":bpm,
        "mood":mood,
        "is_system": True})
    with open("features/" + audio.split("/")[1] + ".json", 'w') as f:
        json.dump(features.flatten().tolist(), f)
        f.close()
    del spectrogram, mood, bpm, features
    gc.collect()
    i+=1
    print(str(i)+"/"+str(len(audios)))

FileNotFoundError: [Errno 2] No such file or directory: 'features/Love Me Now_11.json'

In [ ]:
import pandas as pd
df = pd.DataFrame(data)
df.to_csv("music.csv", index=False, header=True)

In [3]:
import pandas as pd
mood_list = ["Anxious","Chill","Focus","Party","Romance","Sad"]
for mood in mood_list:
    data = json.load(open(mood + ".json"))
    df = pd.DataFrame(data)
    df.to_csv(mood + ".csv", index=False, header=True)